# [CptS 215 Data Analytics Systems and Algorithms](https://piazza.com/wsu/fall2017/cpts215/home)
[Washington State University](https://wsu.edu)

[Gina Sprint](http://eecs.wsu.edu/~gsprint/)
# L8-1 Hashing

Learner objectives for this lesson:
* Understand hash tables and hash functions
* Properly handle collisions
    * Linear probing
    * Chaining


## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Miller and Ranum](http://interactivepython.org/runestone/static/pythonds/index.html)

## Hashing
Let's return to our discussion of searching. The purpose of searching is to find something you have stored somewhere. We have seen different searching algorithms that can perform this task. For example, linear search can find an item in $\mathcal{O}(N)$ time and binary search can find an item in $\mathcal{O}(log(N))$ time. There is another searching technique, called *hashing*, that supports $\mathcal{O}(1)$ time for searching. Hashing typically sacrifices space to improve time complexity. To do this, hashing utilizes information about where an item "should be" in order to look up the item's location. 

A *hash table* is a collection of items that are stored in such a way as to make it easy to find them later. We have actually already seen hashing in action. Does this make you think of any built-in Python data structures we have worked with? Dictionaries! A dictionary is a list of key-value pairs where the keys are indices and do not have to be integers. A hash table is also called a dictionary, a map, a look up table, and an associative array. 

Thinking of a hash table and a dictionary as a map also helps to understand the process of hashing. We can think of a dictionary *mapping* a key to value, that is, there is a function $f$, such that $f(key) = value$. A hash table uses this special function $f$ to compute the location of value by using the value as the key, instead of storing the keys in the table. $f$ is called the $hash function* of the hash table. 

### Example
Let's take a look at an example to pull together the concepts of a hash table and a hash function. Consider the following hash table of size $m=11$, meaning the hash table contains 11 *slots*, or positions:

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|None|None|None|None|None|None|None|None|None|None|None|

Initially, all slots in the table are empty (using the Python reserved keyword `None` to denote empty). Each slot can hold an item and each slot is indexed by an integer value starting at 0. To determined the mapping between an item and the slot where an item belongs in the hash table, we will use a hash function $f(item) = slot$. The hash function will take item in the collection and return an integer representing a slot, e.g. an integer in [0, $m$ - 1]. 

A simple hash function is called the *remainder method*. The remainder method uses mod to determine the location of an item, i.e. its slot: $f(item) = item % m = slot$. 

Suppose we want to insert the value 42 into our hash table. Since $m$ = 11, $f(42) = 42 \% 11 = 9$, we will place 42 at slot 9 in our table:

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|None|None|None|None|None|None|None|None|None|42|None|

Now let's insert a few more items into our table. Namely, we will insert 22, 7, 16, and 90 into the table *in that order*. 

$f(22) = 22 \% 11 = 0$

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|22|None|None|None|None|None|None|None|None|42|None|

$f(7) = 7 \% 11 = 7$

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|22|None|None|None|None|None|None|7|None|42|None|

$f(16) = 16 \% 11 = 5$

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|22|None|None|None|None|16|None|7|None|42|None|

$f(90) = 90 \% 11 = 2$

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|22|None|90|None|None|16|None|7|None|42|None|

Note: Now 5 of the 11 slots are now occupied. The number of slots filled compared to how many slots are available is called the *load factor*, and is commonly denoted by:

$$\lambda = \frac{numberofitems}{m}$$

For our running example, $\lambda=\frac{5}{11}$

### Slot Lookup Efficiency
Now when we want to search for an item, we use the hash function $f$ to determine the slot for the item and then check the hash table to see if it is present. Since a constant amount of time is required to compute the hash value and then index the hash table at that location, this searching operation is $\mathcal{O}(1)$. If everything is where it should be, we have found a constant time search algorithm. 

### Handling Collisions
Will every item always be where it should be? Do you see any potential problems with this approach? What happens if we next insert 35 into the hash table?

$f(35) = 35 \% 11 = 2$

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|22|None|COLLISION:90|None|None|16|None|7|None|42|None|

Item 90 already maps to slot 2 so we have a collision! This approach only works when each item maps to a unique slot. How can hash tables handle multiple items hashing to the same slot?

#### Linear Probing
We can process a collision by simply placing the item in the next available slot in the hash table. We can sequentially search (by re-hashing) until we find an empty slot. If we reach the end of the hash table without finding a slot, the search will continue in a circular manner to the front of the hash table. 

Returning to our example:

$f(35) = 35 \% 11 = 2$

Place 35 in the next available slot (3) instead of the already occupied slot (2).

|0|1|2|3|4|5|6|7|8|9|10|
|-|-|-|-|-|-|-|-|-|-|-|
|22|None|90|35|None|16|None|7|None|42|None|

When retrieving an item from the table, we will also have to "linearly probe" to find the item. Suppose we are now trying to retrieve item 35. We would start at slot 2. If 35 is not in slot 2 (and we know a collision has occurred with this slot, perhaps due to a flag), then we must sequentially search until we find 35 or we have searched the entire list (inefficient! see the suggested improvement below using a slot "flag").

There are a few issues with the current implementation of linear probing. If many collisions occur at the same hash value (slot), then the surrounding slots will be filled due to linear probing. This can cause *clustering* in the hash table; that is, regions of the table that are full and regions that are empty. Ideally we would like an even spread of full vs. empty slots. To help reduce clustering, we can add skip values when rehashing to spread out the insertion of the item from its original slot. Lastly, to address the full sequential search of an item on retrieval, we can add a flag to each slot representing whether the slot has always been empty or if it is empty after removal. The flag helps to optimize item retrieval, because we do not have to search until we reach the item or an empty slot. If the slot has always been empty, we do not need to search beyond this slot. 

#### Chaining (Linked Processing)
An alternative to linear probing is *chaining*, where each slot in the hash table stores a chain of items. In this manner, a single slot can "store" many items. When an an item maps to an already occupied slot (e.g. a collision has occurred), the item is added to a linked list whose head is stored at that slot.

Returning to our example:

$f(35) = 35 \% 11 = 2$

Place 35 in the next available slot (3) instead of the already occupied slot (2). To display the hash table, I've oriented it vertically in order to show the linked lists "growing" to the right of a slot:

|Slot|Linked List|
|-|-|
|0|22|
|1|None|
|2|90|35|
|3|None|
|4|None|
|5|16|
|6|None|
|7|7|
|8|None|
|9|42|
|10|None|

And now let's also add 2:

$f(2) = 2 \% 11 = 2$

|Slot|Linked List|
|-|-|
|0|22|
|1|None|
|2|90|35|2|
|3|None|
|4|None|
|5|16|
|6|None|
|7|7|
|8|None|
|9|42|
|10|None|

When retrieving an item from the table, we will have to search the linked list to find the item to retrieve. The list of items should be short, depending on several factors!

## Analysis of Hashing
If the load factor $\lambda$ is small, meaning the table is fairly empty, there will be a lower chance of collisions occurring. If the load factor $\lambda$ is large, meaning the table is fairly full, there will be a higher chance of collisions occurring. The more collisions there are, the less efficient item placement and retrieval will be (think sequential searching the hash table as in linear probing, and searching the linked list as in chaining). 

### Linear Probing
For successful retrieval using linear probing, the average number of comparisons is: 

$$\frac{1}{2}(1 + \frac{1}{1 - \lambda})$$

For unsuccessful retrieval using linear probing, the average number of comparisons is: 

$$\frac{1}{2}(1 + (\frac{1}{1 - \lambda})^{2})$$

### Chaining
For successful retrieval using chaining, the average number of comparisons is: 

$$1 + \frac{\lambda}{2}$$

For unsuccessful retrieval using linear probing, the average number of comparisons is $\lambda$

## Practice Problems
Note: problems are adapted from Koffman and Wolfgang. 

### 1
For the following items and their hash codes, insert the items (in the order listed in the table) into a hash table of size:
1. 5
1. 7
1. 13

|City|Hash code|Hash code % 5|Hash code % 7|Hash code % 13|
|-|-|-|-|-|
|"Pullman"|84274||||
|"Moscow"|2129869||||
|"Spokane"|69496448||||
|"Seattle"|82879||||
|"Lewiston"|2484038|||||

Use linear probing to handle collisions. Show what the table looks like after inserting all five items.

### 2
Re-solve the previous problem using chaining instead of linear probing. How do the two tables compare?

### 3
Explain what is wrong with the following strategy to reclaim space that is filled with deleted items in a hash table:
>When attempting to insert a new item in the table, if you encounter an item that has been deleted, replace the deleted item with the new item

### 4
Compare the storage requirement for a hash table with linear probing, a table size of 500, and a load factor of 0.5 with a hash table that uses chaining and gives the same performance.

## Summary
* Hashing is appropriate when you have a collection of data and you do not care about its order.
* Hashing can allow for $\mathcal{O}(1)$ placement and retrieval of data.
* Hashing utilizes a hash table (which is usually an array/list)
    * Hash table sizes are usually prime numbers
* To place data into a hash table, you use a hash function
* The hash function:
    * Must produce an integer index into the hash table [0, $m$ - 1] where $m$ is the size of the has table (number of slots)
        * The index produced is based on the data you want to place
        * A perfect hash function produces a unique index for each unique data item
    * That cannot guarantee unique indices will cause collisions
        * Your hash table must handle collisions for placement and retrieval
    * Must always produce the same index because the hash function is also used for data retrieval
* Collisions ruin $\mathcal{O}(1)$, but we must deal with them.
    * Linear probing
    * Chaining (linked processing)
* Hashing sacrifices memory for speed
    * To possibly lessen the chance of collisions, we can use a larger table